# Modelo Xgboost 1 

In [1]:
import common.common_machine_learning as common
import common.metrica as metrica

In [2]:
import numpy as np

from scipy.stats import uniform, randint

from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

import xgboost as xgb

### Ignorar Future Warnings 

In [ ]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

## Constantes 

In [3]:
TARGET = 'precio'
TEST_FRACCION = 0.25
RANDOM_SEMILLA = 1
XGB_OBJECTIVE = 'reg:squarederror'

In [4]:
def display_scores(scores):
    print("Scores: {0}\nMean: {1:.3f}\nStd: {2:.3f}".format(scores, np.mean(scores), np.std(scores)))

In [5]:
def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

## Set de datos 

In [6]:
train = common.cargar_set_optimizado('sets_de_datos/train.csv')

 Descartamos las siguientes columnas para este modelo inicial:

- titulo, descripcion, direccion : strings complejas que requieren simplificacion previa a encodign.
- lat, lng : valores numericos que cuyos nulls no se pueden inputar por valores con sentido de forma simple
- idzona, fecha : mismo motivos que lat, lng

In [7]:
train = common.eliminar_columnas_complejas(train)

In [8]:
train.shape

(240000, 16)

In [9]:
X = train.drop([TARGET], axis = 1).copy()
y = train[TARGET].copy()

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = TEST_FRACCION, random_state = RANDOM_SEMILLA)

In [11]:
X_train = common.label_encode_strings_simples(X_train)
X_test = common.label_encode_strings_simples(X_test)

## Tunneo de hiperparametros

Busqueda 0

In [17]:
4**8

65536

In [19]:
xgb_model = xgb.XGBRegressor(objective = "reg:squarederror")

params = {
    "n_estimators": [100, 250, 750, 1000], # default 100,
    "learning_rate": [0.03, 0.01, 0.1, 0.3], # default 0.1
    "gamma": [0, 0.25, 0.75, 1],
    "lambda": [0, 0.25, 0.75, 1],
    "max_depth": [3, 5, 7, 9], # default 3 # < 15 columnas = cantidad columnas feature originales
    "subsample": [0.6, 0.5, 0.4, 0.3],
    "colsample_bytree": [0.6, 0.5, 0.4, 0.3],
    "min_child_weight": [2, 3, 4, 5]
}

search = RandomizedSearchCV(
            xgb_model, 
            param_distributions=params,
            random_state = RANDOM_SEMILLA,
            n_iter = 200,
            cv=3, 
            verbose=1, 
            n_jobs=1, 
            return_train_score=True,
            iid = True
        )

search.fit(X_train, y_train)

report_best_scores(search.cv_results_, 1)

Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/home/soporte/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/soporte/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/soporte/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/soporte/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/soporte/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base

KeyboardInterrupt: 

Evaluacion